# **Numerical Solution of the Schrödinger Equation for 1D Quantum Well**


### [Go back to index](./index.ipynb)

**Source code:** https://github.com/osscar-org/quantum-mechanics/blob/develop/notebook/1quantumwell.ipynb

<hr style="height:1px;border:none;color:#cccccc;background-color:#cccccc;" />

<p style="text-align: justify;font-size:15px">  
    This notebook displays interactively the eigenfunctions (plotted at the height of the corresponding
    eigenvalues) for a single one-dimensional quantum well, as obtained from the numerical solution.
    There are three sliders, which can be tuned. Two of them can adjust the width and depth of the
    potential well. The zoom factor slider is employed to view the wavefunctions. By increasing the zoom
    factor value, the amplitude of the wavefunctions increases accordingly.
</p>

<p style="text-align: justify;font-size:15px"> 
    There are two subplots as shown below. The wide figure on the left shows the well potential 
    and the square of the eigenfunctions $\psi^2$of different states. 
    The narrow figure on the right shows the eigenvalues.
    One can pick the square of the eigenfunction $\psi^2$ and its eigenvalue by clicking on the plot. 
    The eigenfunction and corresponding eigenvalue will be highlighted by the clicked y-axis data.
    All the other states will be hidden from the plot. By clicking the "Show all" button, one 
    can see all states again.
</p>

<p style="text-align: justify;font-size:15px"> 
    Please tun the sliders to study how the width and depth of the well potential influences the 
    solution of the Schrödinger equation. 
</p>

<hr style="height:1px;border:none;color:#cccccc;background-color:#cccccc;" />

In [ ]:
%matplotlib widget

from numpy import linspace, sqrt, ones, arange, diag, argsort, zeros
from scipy.linalg import eigh_tridiagonal
import matplotlib.pyplot as plt


colors = ['#a6cee3','#1f78b4','#b2df8a','#33a02c','#fb9a99','#e31a1c','#fdbf6f','#ff7f00','#cab2d6','#6a3d9a','#ffff99','#b15928']
ixx = 0

def singlewell_potential(x, width, depth):
    x1 = zeros(len(x))
    for i in range(len(x)):
        if x[i] > - width/2.0 and x[i] < width/2.0:
            x1[i] = depth
    return x1
    

def diagonalisierung(hquer, L, N, pot=singlewell_potential, width = 0.1, depth = 0.0):
    """Calculated sorted eigenvalues and eigenfunctions. 

       Input:
         hquer: Planck constant
         L: set viewed interval [-L,L] 
         N: number of grid points i.e. size of the matrix 
         pot: potential function of the form pot
         x0: center of the quantum well
         width: the width of the quantum well
         depth: the depth of the quantum well
       Ouput:
         ew: sorted eigenvalues (array of length N)
         ef: sorted eigenfunctions, ef[:,i] (size N*N)
         x:  grid points (arry of length N)
         dx: grid space
         V:  Potential at positions x (array of length N)
    """
    x = linspace(-L, L, N+2)[1:N+1]                 # grid points
    dx = x[1] - x[0]                                # grid spacing
    V = pot(x, width, depth)
    z = hquer**2 /2.0/dx**2                         # second diagonals

    ew, ef = eigh_tridiagonal(V+2.0*z, -z*ones(N-1))
    ew = ew.real                                    # real part of the eigenvalues
    ind = argsort(ew)                               # Indizes f. sort. Array
    ew = ew[ind]                                    # Sort the ew by ind
    ef = ef[:, ind]                                 # Sort the columns 
    ef = ef/sqrt(dx)                                # Correct standardization 
    return ew, ef, x, dx, V


def plot_eigenfunktionen(ax, ew, ef, x, V, width=1, Emax=0.05, fak= 5.0):
    """Plot of the lowest eigenfunctions 'ef' in the potential 'V (x)'
       at the level of the eigenvalues 'ew' in the plot area 'ax'.
    """
    
    fak = fak/100.0;
    
    ax[0].axhspan(0.0, Emax, facecolor='lightgrey')
    
    ax[0].set_xlim([min(x), max(x)])
    ax[0].set_ylim([min(V)-0.05, Emax])
    
    ax[0].set_xlabel(r'$x/a$', fontsize = 10)
    ax[0].set_ylabel(r'$V(x)/V_0\ \rm{, Eigenfunctions\ with\ Eigenvalues}$', fontsize = 10)
    
    ax[1].set_xlim([min(x), max(x)])
    ax[1].set_ylim([min(V)-0.05, Emax])
    
    ax[1].yaxis.set_label_position("right")
    ax[1].yaxis.tick_right()
    
    ax[1].get_xaxis().set_visible(False)
    ax[1].set_ylabel(r'$\rm{\ Eigenvalues}$', fontsize = 10)
    
    indmax = sum(ew<=0.0)                       
    if not hasattr(width, "__iter__"):           
        width = width*ones(indmax)               
    for i in arange(indmax):                     
        ax[0].plot(x, fak*abs(ef[:, i])**2+ew[i], linewidth=width[i]+.1, color=colors[i%len(colors)])
        ax[1].plot(x, x*0.0+ew[i], linewidth=width[i]+2.5, color=colors[i%len(colors)])
        
    ax[0].plot(x, V, c='k', linewidth=1.6)


In [ ]:
from ipywidgets import FloatSlider, jslink, VBox, HBox, Button, Label

mu = 0.06                                            # Potential parameter
L = 1.5                                              # x range [-L,L]
N = 200                                              # Number of grid points
hquer = 0.06                                         # Planck constant
sigma_x = 0.1                                        # Width of the Gaussian function
zeiten = linspace(0.0, 10.0, 400)                    # time


swidth = FloatSlider(value = 1.2, min = 0.1, max = 2.0, description = 'Width: ')
sdepth = FloatSlider(value = -0.2, min = -1.0, max = 0.0, step = 0.05, description = 'Depth: ')
sfak = FloatSlider(value = 5, min = 1.0, max = 10.0, step = 1.0, description = r'Zoom factor: ')
update = Button(description="Show all")

width = 1.2
depth = -0.2
fak = 5.0

ew, ef, x, dx, V = diagonalisierung(hquer, L, N, width = width, depth = depth)
    
fig, ax = plt.subplots(1, 2, figsize=(7,5), gridspec_kw={'width_ratios': [10, 1]})
fig.canvas.header_visible = False
fig.canvas.layout.width = "750px"

fig.suptitle(r'Numerical Solution ($\psi^2$) of the Schrödinger Equation for 1D Quantum Well', fontsize = 12)
plot_eigenfunktionen(ax, ew, ef, x, V)

def on_update_click(b):
    for i in ax[0].lines:
        i.set_alpha(1.0)
    for i in ax[1].lines:
        i.set_alpha(1.0)
    try:
        ann.remove()
        ann1.remove()
    except:
        pass

def on_width_change(change):
    global ew, ef, x, dx, V
    ax[0].lines = []
    ax[1].lines = []
    
    try:
        ann.remove()
        ann1.remove()
    except:
        pass

    ew, ef, x, dx, V = diagonalisierung(hquer, L, N, width = swidth.value, depth = sdepth.value)
    plot_eigenfunktionen(ax, ew, ef, x, V, fak = sfak.value)

def on_depth_change(change):
    global ew, ef, x, dx, V
    ax[0].lines = []
    ax[1].lines = []
    
    try:
        ann.remove()
        ann1.remove()
    except:
        pass

    ew, ef, x, dx, V = diagonalisierung(hquer, L, N, width = swidth.value, depth = sdepth.value)
    plot_eigenfunktionen(ax, ew, ef, x, V, fak = sfak.value)
    
def on_xfak_change(change):
    ax[0].lines = []
    ax[1].lines = []
    
    try:
        ann.remove()
        ann1.remove()
    except:
        pass

    plot_eigenfunktionen(ax, ew, ef, x, V, fak = sfak.value)

def on_press(event):
    global ann, ann1, ixx
    
    ixx = min(enumerate(ew), key = lambda x: abs(x[1]-event.ydata))[0]
    
    for i in range(len(ax[1].lines)):
        ax[0].lines[i].set_alpha(0.1)
        ax[1].lines[i].set_alpha(0.1)
        ax[0].lines[i].set_linewidth(1.1)
        
    ax[0].lines[ixx].set_alpha(1.0)
    ax[1].lines[ixx].set_alpha(1.0)
    ax[0].lines[ixx].set_linewidth(2.0)
    
    try:
        ann.remove()
        ann1.remove()
    except:
        pass
    
    ann = ax[0].annotate(s = 'n = ' + str(ixx+1), xy = (0, ew[ixx]), xytext = (-0.15, ew[ixx]), xycoords = 'data', color='k', size=15)
    ann1 = ax[1].annotate(s = str("{:.3f}".format(ew[ixx])), xy = (0, ew[ixx]), xytext = (-1.2, ew[ixx]+0.005), xycoords = 'data', color='k', size=9)

cid = fig.canvas.mpl_connect('button_press_event', on_press)

swidth.observe(on_width_change, names = 'value')
sdepth.observe(on_depth_change, names = 'value')
sfak.observe(on_xfak_change, names = 'value')
update.on_click(on_update_click)

label1 = Label(value="(click on a state to select it)");

display(HBox([swidth, sdepth]), sfak, HBox([update, label1]))

* **Width:** the width of the quantum well.
* **Depth:** the depth of the quantum well.
* **Zoom factor:** the zoom factor of the eigenfunctions. 